In [ ]:
require 'selenium-webdriver'
require 'date'

puts "Using Selenium WebDriver version #{Selenium::WebDriver::VERSION}"

download_dir_path = File.join(Dir.home, "cua-scrape-#{Date.today.strftime('%F')}")
if !Dir.exists?(download_dir_path) then
  Dir.mkdir(download_dir_path)
end
puts "Using CSV download directory of #{download_dir_path}"
prerun_dir_files = Dir.children(download_dir_path)

profile = Selenium::WebDriver::Firefox::Profile.new
ff_opts = Selenium::WebDriver::Firefox::Options.new(profile: profile)
ff_opts.add_preference('browser.download.dir', download_dir_path)
ff_opts.add_preference('browser.download.useDownloadDir', true)
ff_opts.add_preference('browser.helperApps.neverAsk.saveToDisk', "text/csv")
driver = Selenium::WebDriver.for :firefox, options: ff_opts


In [ ]:
#driver.close

In [ ]:
# Navigate to URL
driver.get 'https://ob.greatsouthernbank.com.au/'


In [ ]:
credentials = IRuby.popup 'enter access credentials' do 
  input :username, label: "Username / member no"
  password :wac, label: "WAC"
  button
end

username_field = driver.find_element(id: 'userName')
# puts("Enter username/member no:")
# username = gets
# username_field.send_keys username
username_field.send_keys credentials[:username]

wac_field = driver.find_element(id: 'wac')
# puts("Enter WAC:")
# user_wac = gets
# wac_field.send_keys user_wac
wac_field.send_keys credentials[:wac]

In [ ]:
login_button = driver.find_element(id: 'login')
login_button.click

In [ ]:
# Account summary is displayed in a frame called 'head4'
# ...but the account smmary is loaded with AJAX >:(
wait = Selenium::WebDriver::Wait.new(:timeout => 4)
# wait till the frame exists
wait.until { driver.find_element(id: 'head4')}
# switch into it
driver.switch_to.frame 'head4'
# wait till the account summary has loaded
wait.until { driver.find_element(id: 'accountsSummaryDash') }

puts "account summary has loaded"

In [ ]:
# the account summary elements are fucking stupid because every link
# for each account has the SAME ID. which is technically illegal HTML.
#
# notes from docs:
#     When using Element#find_element with :xpath, be aware that webdriver
#     follows standard conventions: a search prefixed with “//” will search
#     the entire document, not just the children of this current node.
#     Use “.//” to limit your search to the children of the receiving Element.
accounts_table = driver.find_element(id: 'accountDash')
def find_account_link(accounts_table, account_no)
  raise "invalid account_no of '#{account_no}'" if !(/\d+/ =~ account_no)
  return accounts_table.find_element(xpath: ".//tbody/tr/td/a[contains(text(),'#{account_no}')]")
end

account_form = IRuby.popup 'enter account number' do 
  input :account_number, label: "Account Number (e.g. 12345678)"
  button
end

account_id_initial = account_form[:account_number]

if not account_id_initial.match? /^\d+$/ then
  raise "account number must be sequence of digits!"
end

  account_link = find_account_link(accounts_table, account_id_initial)
account_link.click

# transaction list will now load in the same 'head4' frame.
# wait till it has loaded:
# the transaction list USED to be in <body id="accountStatement" /> but
# it seems they 'upgraded'? the system at one point and changed the ID to
# accountStatementUpgrade
# wait.until { driver.find_element(id: 'accountStatement') }
wait.until { driver.find_element(id: 'accountStatementUpgrade') }
puts "account statement for account \##{account_id_initial} is ready"


current DOM layout:

    html
      body#accountStatementUpgrade
      \>form#transaction
      |   div.navbar.formleftmargin
      |     div.navbar-inner
      |       ul.row-fluid
      |         ul.row-fluid.verticalContainerPosition
      |           li.verticalLeftContainer.control-group.success.ie8padding.relativelyPositioned
      |           li.verticalRightContainer.control-group.success.ie8padding.relativelyPositioned
      |             div#transaction_daterange-main.__destroy__.bancs-comboBox.bancs-comboBoxDestroyed4
      |               a#transaction_daterange-box.transaction_daterange_box
      |                 span#transaction_daterange-input.bancs-comboBox_inputs
      |                 span#transaction_daterange-button.bancs-comboBox_buttons
      \>form#transactionExport
          div#changeZindex1.navbar.formleftmargin.zindex1imp
            div#changeZindex3.navbar-inner.bottomPadding.relativelyPositioned.zindex3imp
              ul#accTxn.row-fluid.relatedAcc
                ul.overrideFluidPadding
                  ul.row-fluid.flyOutMenu.accountDetails.actionsMenu
                  ul.row-fluid.flyOutMenu.accountDetails.exportMenu
                    li#changeClassLoad
                      a#menubutton1.anchor.accountStatementAction.textStyle1
                  ul.row-fluid.verticalContainerPosition
                    li.success.ie8padding
                      div#accountStatement_print a.textStyle1


In [ ]:

# need to change the transaction list to one going back 6 months.
# open the date range dropdown
def change_txn_list_to_six_months(driver)
  daterange_btn = driver.find_element(id: 'transaction_daterange-button')
  daterange_btn.click

  # "The last 6 months" option is IDed as follows...
  six_months_option = driver.find_element(id: 'transaction_daterange-lists_7')
  # log out what the text says just in case
  puts "Selecting 6-months option with text '#{six_months_option.text}'"
  six_months_option.click
end

change_txn_list_to_six_months(driver)

def run_txn_list_report(driver)
  # run the report
  puts "Running report..."
  driver.find_element(id: 'accountStatement_generate').click
  # this might actually take a bit of time, so grant a more generous timeout
  wait = Selenium::WebDriver::Wait.new(:timeout => 10)
  wait.until { driver.execute_script('return document.readyState;') == 'complete' }
  puts "report ready"
end

run_txn_list_report(driver)

In [ ]:
# now do a CSV export
def do_csv_export(driver)
  export_menu = driver.find_element(css: 'ul.exportMenu')
  export_menu.find_element(xpath: './/li[1]/a').click
  export_menu.find_element(xpath: ".//li[1]/ul/li/a[contains(text(),'CSV')]").click
end


In [ ]:
do_csv_export(driver)

In [ ]:
def change_txn_list_account(driver, account_number)
  acct_dropdown_btn = driver.find_element(id: 'transaction_savingsandcurrentaccounts-button')
  acct_dropdown_btn.click
  wait = Selenium::WebDriver::Wait.new(:timeout => 1)
  account_list = wait.until {
    driver.find_element(id: 'transaction_savingsandcurrentaccounts-lists')
  }
  account_list_item = account_list.find_element(xpath: ".//li[contains(text(),'#{account_number}')]")
  account_list_item.click
end

account_form = IRuby.popup 'enter account number' do 
  input :account_number, label: "Account Number (e.g. 12345678)"
  button
end

account_id_second = account_form[:account_number]

if not account_id_second.match? /^\d+$/ then
  raise "account number must be sequence of digits!"
end

  change_txn_list_account(driver, account_id_second)
# changing account causes date range reset, so choose 6 months again
change_txn_list_to_six_months(driver)

run_txn_list_report(driver)

In [ ]:
do_csv_export(driver)